In [131]:
import tensorflow as tf
from sklearn import metrics
import numpy as np
from scipy import stats
import os
import sys
import matplotlib.pyplot as plt
import matplotlib.colors as colors

parent_directory = os.path.abspath('..')
sys.path.append(parent_directory)
from src import data as d

In [3]:
dataLoader = d.DataLoader("../localdata/")
(train_images, train_labels_ix, train_labels, 
           test_images, test_labels_ix, test_labels, 
           shifted_images, shifted_labels_ix, shifted_labels,
           num_classes ) = dataLoader.load_data('CIFAR10')

In [13]:
def compute_stats(metric):
    """
    Compute stat desc of a metric
    """
    stats = dict()
    stats['mean'] = np.mean(metric)
    stats['std'] = np.std(metric)
    stats['min'] = np.min(metric)
    stats['max'] = np.max(metric)
    return stats

# Single models

In [21]:
models_path = "../results/raw/single"

acc = []
iou = []
f1 = []
ind_pred_classes = None
ind_pred_probs = list()
acc_shifted = []
iou_shifted = []
f1_shifted = []
ind_pred_classes_shifted = None
ind_pred_shifted_probs = list()

for f in os.listdir(models_path):
    if f.endswith("npy"):
        with open(os.path.join(models_path, f), 'rb') as npy:
            pred = np.load(npy)
            pred_shifted = np.load(npy)

        ind_pred_probs.append(pred)
        ind_pred_shifted_probs.append(pred_shifted)
        pred_classes = np.argmax(pred, axis=1)
        pred_classes_shifted = np.argmax(pred_shifted, axis=1)
        
        if ind_pred_classes is None:
            ind_pred_classes = pred_classes
        else:
            ind_pred_classes = np.vstack([ind_pred_classes, pred_classes])
            
        if ind_pred_classes_shifted is None:
            ind_pred_classes_shifted = pred_classes_shifted
        else:
            ind_pred_classes_shifted = np.vstack([ind_pred_classes_shifted, pred_classes_shifted])
            
        acc.append(metrics.accuracy_score(test_labels_ix, pred_classes))
        iou.append(metrics.jaccard_score(test_labels_ix, pred_classes, average="weighted"))
        f1.append(metrics.f1_score(test_labels_ix, pred_classes, average="weighted"))
        
        acc_shifted.append(metrics.accuracy_score(shifted_labels_ix, pred_classes_shifted))
        iou_shifted.append(metrics.jaccard_score(shifted_labels_ix, pred_classes_shifted, average="weighted"))
        f1_shifted.append(metrics.f1_score(shifted_labels_ix, pred_classes_shifted, average="weighted"))

In [33]:
stats_acc = compute_stats(acc)
stats_iou = compute_stats(iou)
stats_f1 = compute_stats(f1)

stats_acc_shifted = compute_stats(acc_shifted)
stats_iou_shifted = compute_stats(iou_shifted)
stats_f1_shifted = compute_stats(f1_shifted)

# Deep ensemble

In [35]:
ens_len = 12
ens_set = np.random.permutation(range(len(ind_pred_probs)))[0:ens_len]

ind_pred_classes_ens = ind_pred_classes[ens_set]
ind_pred_classes_shifted_ens = ind_pred_classes_shifted[ens_set]

In [39]:
pred_mode = stats.mode(ind_pred_classes_ens).mode    

pred_mode_shifted = stats.mode(ind_pred_classes_shifted_ens).mode

mode_acc = metrics.accuracy_score(test_labels_ix, pred_mode)
mode_iou = metrics.jaccard_score(test_labels_ix, pred_mode, average="weighted")
mode_f1 = metrics.f1_score(test_labels_ix, pred_mode, average="weighted")

mode_acc_shifted = metrics.accuracy_score(shifted_labels_ix, pred_mode_shifted)
mode_iou_shifted = metrics.jaccard_score(shifted_labels_ix, pred_mode_shifted, average="weighted")
mode_f1_shifted = metrics.f1_score(shifted_labels_ix, pred_mode_shifted, average="weighted")

In [113]:
# variance
tmp = np.stack(ind_pred_probs, axis=1)
var_samp_class = np.var(tmp, axis=1)
var_samp = np.sum(var_samp_class, axis=1)
var_ens = np.mean(var_samp)
    
tmp = np.stack(ind_pred_shifted_probs, axis=1)
var_samp_class = np.var(tmp, axis=1)
var_samp = np.sum(var_samp_class, axis=1)
var_ens_shifted = np.mean(var_samp)

# entropy
l_z = None
for i in range(ind_pred_classes.shape[0]):
    if l_z is None:
        l_z = (ind_pred_classes[i,:] == test_labels_ix[:,0]) * 1
    else:
        l_z = l_z + (ind_pred_classes[i,:] == test_labels_ix[:,0]) * 1

entropy_ens = np.mean(np.minimum(l_z, ind_pred_classes.shape[0] - l_z) / (ind_pred_classes.shape[0] - np.ceil(ind_pred_classes.shape[0]/2)))
    
l_z = None
for i in range(ind_pred_classes_shifted.shape[0]):
    if l_z is None:
        l_z = (ind_pred_classes_shifted[i,:] == shifted_labels_ix[:,0]) * 1
    else:
        l_z = l_z + (ind_pred_classes_shifted[i,:] == shifted_labels_ix[:,0]) * 1

entropy_ens_shifted = np.mean(np.minimum(l_z, ind_pred_classes_shifted.shape[0] - l_z) / (ind_pred_classes_shifted.shape[0] - np.ceil(ind_pred_classes_shifted.shape[0]/2)))

In [114]:
print("Performance single model")
print("Normal\tmean\tstd\tmin\tmax")
print("acc\t", "\t".join(["%5.4f" % n for n in stats_acc.values()]))
print("iou\t", "\t".join(["%5.4f" % n for n in stats_iou.values()]))
print("f1\t", "\t".join(["%5.4f" % n for n in stats_f1.values()]))
print("\nShifted\tmean\tstd\tmin\tmax")
print("acc\t", "\t".join(["%5.4f" % n for n in stats_acc_shifted.values()]))
print("iou\t", "\t".join(["%5.4f" % n for n in stats_iou_shifted.values()]))
print("f1\t", "\t".join(["%5.4f" % n for n in stats_f1_shifted.values()]))

print("\nPerformance ensemble (normal vs shifted)")
print("Acc: \t%5.4f vs %5.4f" % (mode_acc, mode_acc_shifted))
print("IoU: \t%5.4f vs %5.4f" % (mode_iou, mode_iou_shifted))
print("F1: \t%5.4f vs %5.4f" % (mode_f1, mode_f1_shifted))
    
    
print("\n\nVar ensemble %5.4f" % var_ens)
print("Entropy ensemble %5.4f" % entropy_ens)
print("\n\nVar ensemble shifted %5.4f" % var_ens_shifted)
print("Entropy ensemble shifted %5.4f" % entropy_ens_shifted)

Performance single model
Normal	mean	std	min	max
acc	 0.8675	0.0047	0.8604	0.8759
iou	 0.7692	0.0076	0.7579	0.7823
f1	 0.8664	0.0050	0.8590	0.8751

Shifted	mean	std	min	max
acc	 0.7570	0.0091	0.7402	0.7734
iou	 0.6097	0.0121	0.5875	0.6307
f1	 0.7552	0.0094	0.7382	0.7715

Performance ensemble (normal vs shifted)
Acc: 	0.8940 vs 0.7902
IoU: 	0.8110 vs 0.6534
F1: 	0.8934 vs 0.7888


Var ensemble 0.0439
Entropy ensemble 0.1221


Var ensemble shifted 0.0688
Entropy ensemble shifted 0.1788


# MC Dropout

https://www.tensorflow.org/tutorials/understanding/sngp

In [75]:
model_names = [m for m in os.listdir(models_path) if m.endswith("keras") ]
index = 0
print(model_names[index])
model = tf.keras.models.load_model(os.path.join(models_path, model_names[index]))

In [112]:
mc_preds = [model(test_images, training=True) for _ in range(ens_len)]

In [115]:
mc_pred_classes = None
for pred in mc_preds:
    pred_classes = np.argmax(pred, axis=1)
    if mc_pred_classes is None:
        mc_pred_classes = pred_classes
    else:
        mc_pred_classes = np.vstack([mc_pred_classes, pred_classes])

In [116]:
mc_pred_ens = tf.reduce_mean(mc_preds, axis=0)
mc_pred_ens_classes = np.argmax(mc_pred_ens, axis=1)

mc_acc = metrics.accuracy_score(test_labels_ix, mc_pred_ens_classes)
mc_iou = metrics.jaccard_score(test_labels_ix, mc_pred_ens_classes, average="weighted")
mc_f1 = metrics.f1_score(test_labels_ix, mc_pred_ens_classes, average="weighted")

In [117]:
# variance
tmp = np.stack(mc_preds, axis=1)
var_samp_class = np.var(tmp, axis=1)
var_samp = np.sum(var_samp_class, axis=1)
var_mc = np.mean(var_samp)
    

# entropy
l_z = None
for i in range(mc_pred_classes.shape[0]):
    if l_z is None:
        l_z = (mc_pred_classes[i,:] == test_labels_ix[:,0]) * 1
    else:
        l_z = l_z + (mc_pred_classes[i,:] == test_labels_ix[:,0]) * 1

entropy_mc = np.mean(np.minimum(l_z, mc_pred_classes.shape[0] - l_z) / 
                      (mc_pred_classes.shape[0] - np.ceil(ind_pred_classes.shape[0]/2)))


In [118]:
print("\nPerformance deep ensemble vs MC")
print("Acc: \t%5.4f vs %5.4f" % (mode_acc, mc_acc))
print("IoU: \t%5.4f vs %5.4f" % (mode_iou, mc_iou))
print("F1: \t%5.4f vs %5.4f" % (mode_f1, mc_f1))
print("Var: \t%5.4f vs %5.4f" % (var_ens, var_mc))
print("Entropy: \t%5.4f vs %5.4f" % (entropy_ens, entropy_mc))



Performance deep ensemble vs MC
Acc: 	0.8940 vs 0.8702
IoU: 	0.8110 vs 0.7740
F1: 	0.8934 vs 0.8699
Var: 	0.0439 vs 0.0734
Entropy: 	0.1221 vs 0.2805


In [153]:
print(model_names[0])

h3107665801345511179_model.keras
